In [1]:
import pandas as pd
import numpy as np
import urllib.request as urllib2
from datetime import datetime, timedelta, date

### Get Oxford data and calculate Death Rate

In [2]:
# Import Oxford data and enrich columns for future merging needs
df = pd.read_csv('https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv')
df['Date'] = pd.to_datetime(df['Date'], format="%Y%m%d")
df['DateString'] = df['Date'].apply(lambda x: datetime.strftime(x, "%Y%m%d"))
df['Index'] = df['CountryCode'] + df['DateString']

# Calculate Death Rates
df['DeathRate'] = df['ConfirmedDeaths'] / df['ConfirmedCases'] *100
df

,CountryName,CountryCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,...,M1_Wildcard,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,LegacyStringencyIndex,LegacyStringencyIndexForDisplay,DateString,Index,DeathRate
0,Aruba,ABW,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,0.00,0.00,0.00,0.00,20200101,ABW20200101,NaN
1,Aruba,ABW,2020-01-02,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,0.00,0.00,0.00,0.00,20200102,ABW20200102,NaN
2,Aruba,ABW,2020-01-03,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,0.00,0.00,0.00,0.00,20200103,ABW20200103,NaN
3,Aruba,ABW,2020-01-04,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,0.00,0.00,0.00,0.00,20200104,ABW20200104,NaN
4,Aruba,ABW,2020-01-05,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,0.00,0.00,0.00,0.00,20200105,ABW20200105,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21853,Kosovo,RKS,2020-05-09,3.0,1.0,2.0,1.0,2.0,1.0,4.0,...,NaN,862.0,28.0,94.71,94.71,93.57,93.57,20200509,RKS20200509,3.248260
21854,Kosovo,RKS,2020-05-10,3.0,1.0,2.0,1.0,2.0,1.0,4.0,...,NaN,870.0,28.0,94.71,94.71,93.57,93.57,20200510,RKS20200510,3.218391
21855,Kosovo,RKS,2020-05-11,3.0,1.0,2.0,1.0,2.0,1.0,4.0,...,NaN,884.0,28.0,94.71,94.71,93.57,93.57,20200511,RKS20200511,3.167421
21856,Kosovo,RKS,2020-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,919.0,29.0,NaN,94.71,NaN,93.57,20200512,RKS20200512,3.155604


### Get Market data and calculate Market Performance

In [3]:
stocks = pd.read_csv('https://raw.githubusercontent.com/LarsBryld/socialdataanalysis2020/master/data/Markets.csv',
                     encoding= 'unicode_escape'
                    )
stocks = stocks.groupby('CountryCode',sort=False).sum()
#stocks = stocks.rename({'United Kingdom': 'United Kingdom'}, axis='index')
#stocks = stocks.drop(columns=['Index ', 'Country/Region'])

columns_new = np.array(stocks.columns)
for i in range(len(columns_new)):
    columns_new[i] = datetime.strptime(columns_new[i],'%d/%m/%Y')

stocks.columns = columns_new

stocks#.loc['United Kingdom']                                                    # TO BE DELETED

,2020-01-02,2020-01-03,2020-01-04,2020-01-05,2020-01-06,2020-01-07,2020-01-08,2020-01-09,2020-01-10,2020-01-11,...,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15
CountryCode,,,,,,,,,,,,,,,,,,,,,
CHN,3066,3089,3083,3077,3071,3085,3094,3083,3102,3099,...,2797,2807,2806,2826,2827,2813,2799,2785,2795,2827
AUT,3188,3230,3225,3220,3215,3202,3190,3196,3204,3209,...,1966,2050,2127,2115,2129,2143,2157,2170,2184,2173
BEL,3983,4002,3988,3974,3961,3982,3966,4019,4025,4025,...,2965,3036,3029,3106,3111,3116,3121,3126,3131,3045
DNK,1254,1252,1247,1242,1237,1240,1240,1241,1247,1247,...,1155,1165,1155,1158,1161,1164,1167,1170,1173,1185
FIN,4225,4282,4278,4274,4271,4266,4255,4244,4240,4247,...,3326,3428,3419,3482,3492,3502,3512,3522,3532,3532
FRA,6017,6008,6006,6003,6001,6033,5987,6067,6057,6051,...,4299,4489,4397,4512,4521,4529,4537,4545,4554,4512
ITA,23607,23710,23662,23614,23566,23688,23551,23980,24061,24061,...,16769,17588,17212,17686,17713,17740,17767,17795,17822,17455
DEU,13234,13266,13206,13146,13085,13200,13140,13475,13533,13530,...,9889,10464,10302,10491,10539,10588,10637,10685,10734,10678
GRC,918,926,924,922,920,917,916,920,927,926,...,546,590,606,611,611,610,610,610,610,626


In [4]:
# Reference dates: 23rd and 24th March 2020

market0 = stocks[datetime(2020, 3, 23, 0, 0)]
for i in (8, 9, 10, 16, 17, 20, 21, 27, 29, 30, 31, 32):                            #range(1,len(stocks)):
    market0.iloc[i] = stocks[datetime(2020, 3, 24, 0, 0)][i]

market0.head()         # TO BE DELETED

CountryCode
CHN    2678
AUT    1888
BEL    2676
DNK     989
FIN    2939
Name: 2020-03-23 00:00:00, dtype: int64

In [5]:
# Market Performance
mkt_p = stocks.T/market0 -1
mkt_p = mkt_p.T *100
#mkt_p = mkt_p.reindex(focuscountries)

# showing perfomances from 23rd March onward
mkt_p.iloc[:, 81::].head()        # TO BE DELETED

,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,...,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15
CountryCode,,,,,,,,,,,,,,,,,,,,,
CHN,0.0,0.933532,3.622106,3.136669,4.294249,3.622106,2.949963,2.315161,3.323376,2.464526,...,4.443615,4.817028,4.779686,5.526512,5.563854,5.041075,4.518297,3.995519,4.368932,5.563854
AUT,0.0,-0.317797,7.786017,6.885593,8.951271,7.944915,6.938559,5.985169,5.402542,5.932203,...,4.131356,8.580508,12.658898,12.023305,12.764831,13.506356,14.247881,14.936441,15.677966,15.095339
BEL,0.0,4.446936,11.397608,6.614350,9.566517,8.594918,7.623318,6.651719,7.025411,6.016442,...,10.799701,13.452915,13.191330,16.068759,16.255605,16.442451,16.629297,16.816143,17.002990,13.789238
DNK,0.0,3.741153,8.088979,6.673407,9.201213,9.302326,9.504550,9.706775,14.863498,12.436805,...,16.784631,17.795753,16.784631,17.087968,17.391304,17.694641,17.997978,18.301314,18.604651,19.817998
FIN,0.0,2.483838,9.663151,9.118748,11.976863,11.194284,10.411705,9.629126,11.738687,11.500510,...,13.167744,16.638312,16.332086,18.475672,18.815924,19.156176,19.496427,19.836679,20.176931,20.176931


In [6]:
# Converting Market Performances data from rows to columns
mkt_p1 = pd.DataFrame(mkt_p[mkt_p.columns[0]])
mkt_p1['Date'] = mkt_p1.columns.repeat(len(mkt_p1))
mkt_p1.rename(columns={mkt_p1.columns[0]:'MarketPerformance', 'Date':'Date'}, inplace=True)

for i in range(1, len(mkt_p.columns)):
    mkt_p_new = pd.DataFrame(mkt_p[mkt_p.columns[i]])
    mkt_p_new['Date'] = mkt_p_new.columns.repeat(len(mkt_p))
    mkt_p_new.rename(columns={mkt_p_new.columns[0]:'MarketPerformance', 'Date':'Date'}, inplace=True)
    mkt_p1 = pd.concat([mkt_p1, mkt_p_new], axis=0)

mkt_p1.head()

,MarketPerformance,Date
CountryCode,,
CHN,14.488424,2020-01-02
AUT,68.855932,2020-01-02
BEL,48.841555,2020-01-02
DNK,26.794742,2020-01-02
FIN,43.756380,2020-01-02


In [7]:
# Adding columns needed for merging with Oxford data
mkt_p1['DateString'] = mkt_p1['Date'].apply(lambda x: datetime.strftime(x, "%Y%m%d"))
mkt_p1['Index'] = mkt_p1.index + mkt_p1['DateString']
mkt_p1.loc['USA']

,MarketPerformance,Date,DateString,Index
CountryCode,,,,
USA,41.641205,2020-01-02,20200102,USA20200102
USA,40.811873,2020-01-03,20200103,USA20200103
USA,40.680925,2020-01-04,20200104,USA20200104
USA,40.549978,2020-01-05,20200105,USA20200105
USA,40.462680,2020-01-06,20200106,USA20200106
...,...,...,...,...
USA,21.344391,2020-04-11,20200411,USA20200411
USA,21.388040,2020-04-12,20200412,USA20200412
USA,21.431689,2020-04-13,20200413,USA20200413


In [8]:
# Add Market Performance to Oxford file 
df = df.merge(mkt_p1, left_on = 'Index', right_on = 'Index', how = 'left')
df = df.rename({'Date_x': 'Date',
                'DateString_x': 'DateString'}, axis='columns')
df = df.drop(columns=['Date_y', 'DateString_y'])
df[df['CountryCode']=='USA']

,CountryName,CountryCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,...,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,LegacyStringencyIndex,LegacyStringencyIndexForDisplay,DateString,Index,DeathRate,MarketPerformance
20509,United States,USA,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,20200101,USA20200101,NaN,NaN
20510,United States,USA,2020-01-02,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,20200102,USA20200102,NaN,41.641205
20511,United States,USA,2020-01-03,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,20200103,USA20200103,NaN,40.811873
20512,United States,USA,2020-01-04,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,20200104,USA20200104,NaN,40.680925
20513,United States,USA,2020-01-05,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,20200105,USA20200105,NaN,40.549978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20639,United States,USA,2020-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1309541.0,78794.0,NaN,71.58,NaN,73.57,20200510,USA20200510,6.016917,NaN
20640,United States,USA,2020-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1329799.0,79528.0,NaN,71.58,NaN,73.57,20200511,USA20200511,5.980453,NaN
20641,United States,USA,2020-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1347916.0,80684.0,NaN,71.58,NaN,73.57,20200512,USA20200512,5.985833,NaN
20642,United States,USA,2020-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1369964.0,82387.0,NaN,71.58,NaN,73.57,20200513,USA20200513,6.013808,NaN


In [9]:
#data = df

## calculate CovIndex = Market Performance - Death Rate

In [10]:
df['CovIndex'] = (df['MarketPerformance'] - df['DeathRate']) #* 100
df = df[df['CovIndex'].notna()]
df[df['CountryCode']=='USA']

,CountryName,CountryCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,...,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,LegacyStringencyIndex,LegacyStringencyIndexForDisplay,DateString,Index,DeathRate,MarketPerformance,CovIndex
20529,United States,USA,2020-01-21,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.00,0.00,0.00,20200121,USA20200121,0.000000,44.958533,44.958533
20530,United States,USA,2020-01-22,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.00,0.00,0.00,20200122,USA20200122,0.000000,45.351375,45.351375
20531,United States,USA,2020-01-23,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.00,0.00,0.00,20200123,USA20200123,0.000000,44.740288,44.740288
20532,United States,USA,2020-01-24,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.00,0.00,0.00,20200124,USA20200124,0.000000,45.482322,45.482322
20533,United States,USA,2020-01-25,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.00,0.00,0.00,0.00,20200125,USA20200125,0.000000,44.216499,44.216499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20610,United States,USA,2020-04-11,3.0,0.0,3.0,1.0,2.0,0.0,4.0,...,18777.0,71.58,71.58,73.57,73.57,20200411,USA20200411,3.743720,21.344391,17.600672
20611,United States,USA,2020-04-12,3.0,0.0,3.0,1.0,2.0,0.0,4.0,...,20608.0,71.58,71.58,73.57,73.57,20200412,USA20200412,3.888661,21.388040,17.499379
20612,United States,USA,2020-04-13,3.0,0.0,3.0,1.0,2.0,0.0,4.0,...,22108.0,71.58,71.58,73.57,73.57,20200413,USA20200413,3.965056,21.431689,17.466634
20613,United States,USA,2020-04-14,3.0,0.0,3.0,1.0,2.0,0.0,4.0,...,23649.0,71.58,71.58,73.57,73.57,20200414,USA20200414,4.059259,22.435618,18.376359


## Add more info:
- take snapshot o all indicators on 30th April (needed for classifier later on)
- sum all Fiscal and other economic measure up to 30th April
- divide them by GDP (World Bank data are available) to make all Countries measure comparable

## Building the Classifier

In [11]:
good = ['DEU','GRC','USA', 'KOR', 'BRA', 'ZAF', 'DNK']
bad = ['ITA','FRA','SPA', 'GBR', 'BEL', 'SWE', 'NLD']

In [12]:
# data preprocessing

# selecting our sample
# Oxford Data snapsot on 30th March
df1 = df[df['Date'] == datetime(2020, 3, 30, 0, 0)]
df1 = df1.iloc[:,1:-8]
df1 = df1.drop(columns=['Date', 'ConfirmedCases', 'ConfirmedDeaths'])
df1 = df1.fillna(0)

# Labels for classifiers
good = ['DEU','GRC','USA', 'KOR', 'BRA', 'ZAF', 'DNK']
bad = ['ITA','FRA','SPA', 'GBR', 'BEL', 'SWE', 'NLD']

df1['Label'] = 1
df1.loc[df['CountryCode'].isin(good), 'Label'] = 2
df1.loc[df['CountryCode'].isin(bad), 'Label'] = 0

# encoding string labels (Category is the target variable)
labels = df1['Label']

# Select the Features
features = df1.iloc[:,1:-1]

# DecisionTree Classifier
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features, labels)

# Predict Output
# predicted= clf.predict(features) # 0:Overcast, 2:Mild
# print(predicted)

# measuring the classification performance of the Tree classifier through cross_val_score
#from sklearn.model_selection import cross_val_score
#np.mean(cross_val_score(clf, features, label))#, cv=10))

In [13]:
df

,CountryName,CountryCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,...,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,LegacyStringencyIndex,LegacyStringencyIndexForDisplay,DateString,Index,DeathRate,MarketPerformance,CovIndex
969,Australia,AUS,2020-01-25,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,7.94,7.94,9.52,9.52,20200125,AUS20200125,0.000000,55.719314,55.719314
970,Australia,AUS,2020-01-26,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,7.94,7.94,9.52,9.52,20200126,AUS20200126,0.000000,55.521337,55.521337
971,Australia,AUS,2020-01-27,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,7.94,7.94,9.52,9.52,20200127,AUS20200127,0.000000,55.367356,55.367356
972,Australia,AUS,2020-01-28,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,7.94,7.94,9.52,9.52,20200128,AUS20200128,0.000000,55.103388,55.103388
973,Australia,AUS,2020-01-29,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,7.94,7.94,9.52,9.52,20200129,AUS20200129,0.000000,53.871535,53.871535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21690,Taiwan,TWN,2020-04-11,0.0,NaN,0.0,NaN,1.0,1.0,0.0,...,5.0,29.36,29.36,35.24,35.24,20200411,TWN20200411,1.315789,12.109462,10.793672
21691,Taiwan,TWN,2020-04-12,0.0,NaN,0.0,NaN,1.0,1.0,0.0,...,6.0,29.36,29.36,35.24,35.24,20200412,TWN20200412,1.558442,12.275648,10.717207
21692,Taiwan,TWN,2020-04-13,0.0,NaN,0.0,NaN,1.0,1.0,0.0,...,6.0,29.36,29.36,35.24,35.24,20200413,TWN20200413,1.546392,12.430756,10.884364
21693,Taiwan,TWN,2020-04-14,0.0,NaN,0.0,NaN,1.0,1.0,0.0,...,6.0,29.36,29.36,35.24,35.24,20200414,TWN20200414,1.526718,12.242411,10.715693


In [14]:
df1

,CountryCode,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,C5_Close public transport,...,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,M1_Wildcard,StringencyIndex,Label
1034,AUS,3.0,0.0,2.0,0.0,2.0,1.0,4.0,1.0,0.0,...,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,71.44,1
1169,AUT,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,1.0,...,0.0,2.0,1.0,2.0,2.0,0.000000e+00,0.0,0.0,84.79,1
1574,BEL,2.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,0.0,...,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,83.60,0
2924,BRA,3.0,0.0,3.0,0.0,2.0,0.0,3.0,0.0,2.0,...,0.0,2.0,1.0,1.0,0.0,0.000000e+00,0.0,0.0,70.52,2
3599,CAN,3.0,1.0,3.0,0.0,2.0,0.0,4.0,0.0,0.0,...,0.0,2.0,1.0,3.0,1.0,5.620000e+02,191.0,0.0,73.56,1
3734,CHE,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,0.0,...,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,79.49,1
4004,CHN,3.0,0.0,3.0,0.0,2.0,0.0,4.0,0.0,0.0,...,0.0,1.0,1.0,2.0,2.0,0.000000e+00,0.0,0.0,63.11,1
5489,DEU,3.0,1.0,1.0,1.0,2.0,1.0,4.0,1.0,0.0,...,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,74.61,2
5894,DNK,3.0,1.0,2.0,1.0,2.0,1.0,4.0,1.0,1.0,...,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,84.12,2
6569,ESP,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,1.0,...,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,89.41,1


In [15]:
features

,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,C5_Close public transport,C5_Flag,...,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,M1_Wildcard,StringencyIndex
1034,3.0,0.0,2.0,0.0,2.0,1.0,4.0,1.0,0.0,0.0,...,8.022300e+10,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,71.44
1169,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,...,0.000000e+00,0.0,2.0,1.0,2.0,2.0,0.000000e+00,0.0,0.0,84.79
1574,2.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,0.0,0.0,...,0.000000e+00,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,83.60
2924,3.0,0.0,3.0,0.0,2.0,0.0,3.0,0.0,2.0,0.0,...,0.000000e+00,0.0,2.0,1.0,1.0,0.0,0.000000e+00,0.0,0.0,70.52
3599,3.0,1.0,3.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,...,0.000000e+00,0.0,2.0,1.0,3.0,1.0,5.620000e+02,191.0,0.0,73.56
3734,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,0.0,0.0,...,0.000000e+00,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,79.49
4004,3.0,0.0,3.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,...,0.000000e+00,0.0,1.0,1.0,2.0,2.0,0.000000e+00,0.0,0.0,63.11
5489,3.0,1.0,1.0,1.0,2.0,1.0,4.0,1.0,0.0,0.0,...,0.000000e+00,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,74.61
5894,3.0,1.0,2.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,...,0.000000e+00,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,84.12
6569,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,...,0.000000e+00,0.0,2.0,1.0,1.0,1.0,0.000000e+00,0.0,0.0,89.41


In [16]:
labels

1034     1
1169     1
1574     0
2924     2
3599     1
3734     1
4004     1
5489     2
5894     2
6569     1
6974     1
7109     0
7379     0
7919     2
8594     1
9268     1
9403     1
9808     1
9943     1
10078    0
10483    1
11023    2
12774    1
14529    0
14664    1
14799    1
15744    1
16014    1
16689    1
17364    1
18714    0
20598    2
21273    2
21678    1
Name: Label, dtype: int64

In [17]:
#df.to_excel("Dataset.xlsx") 